In [ ]:
import pandas as pd

### local
root_path = '../data/open'

### colab
# root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

### 기본 폴더 구조 설정
data_splits = ["train", "test"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "회원정보": {"folder": "1.회원정보", "suffix": "회원정보", "var_prefix": "customer"},
    "신용정보": {"folder": "2.신용정보", "suffix": "신용정보", "var_prefix": "credit"},
    "승인매출정보": {"folder": "3.승인매출정보", "suffix": "승인매출정보", "var_prefix": "sales"},
    "청구정보": {"folder": "4.청구입금정보", "suffix": "청구정보", "var_prefix": "billing"},
    "잔액정보": {"folder": "5.잔액정보", "suffix": "잔액정보", "var_prefix": "balance"},
    "채널정보": {"folder": "6.채널정보", "suffix": "채널정보", "var_prefix": "channel"},
    "마케팅정보": {"folder": "7.마케팅정보", "suffix": "마케팅정보", "var_prefix": "marketing"},
    "성과정보": {"folder": "8.성과정보", "suffix": "성과정보", "var_prefix": "performance"}
}

months = ["07","08","09","10","11","12"]

In [ ]:
### 선택
split = "train"
select_key = "청구정보"
folder = data_categories[select_key]["folder"]
suffix = data_categories[select_key]["suffix"] 

file_path = file_path = f"{root_path}/{split}/{folder}/cleaned_{suffix}.parquet"
#file_path = file_path = f"{root_path}/{split}/{folder}/201812_processed_{suffix}.parquet"
df = pd.read_parquet(file_path)

In [ ]:
exclude_cols = ['ID', '기준년월', 'Segment_A', 'Segment_B', 'Segment_C', 'Segment_D', 'Segment_E']
target_cols = ['Segment_A', 'Segment_B', 'Segment_C', 'Segment_D', 'Segment_E']

# 1. 타겟 컬럼 제외한 피처들만 추출
feature_cols = [col for col in df.columns if col not in exclude_cols]

# 2. 전체 상관계수 행렬 계산
corr_matrix = df[feature_cols + target_cols].corr()

# 3. 피처-타겟 간 상관계수만 추출
feature_target_corr = corr_matrix.loc[feature_cols, target_cols]

# 4. 절대값이 0.2 이상인 값이 하나라도 있는 피처만 남김
selected_features = feature_target_corr[feature_target_corr.abs().max(axis=1) >= 0.2].index.tolist()

# 5. 해당 피처만 선택한 새로운 데이터프레임 만들기
df_selected = df[selected_features + exclude_cols]  # (타겟도 함께 포함하고 싶다면)

# 선택된 피처 목록만 보고 싶을 땐:
print("선택된 피처 개수:", len(selected_features))
print("선택된 피처들:", selected_features)

In [ ]:
# 저장
#output_file = f"{root_path}/{split}/{folder}/corr_drop_{suffix}.parquet"
output_file = f"{root_path}/{split}/{folder}/201812_corr_drop_{suffix}.parquet"
df_selected.to_parquet(output_file, index=False)
print(f"✅ 저장 완료: {split}/corr_drop_{suffix}")